In [1]:
import time
import csv
import copy
from ipywidgets import interact
import numpy as np
from pandas import pandas as pd

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models.layouts import Column
from bokeh.models import Range1d, DataRange1d
from bokeh.plotting import figure
from bokeh.palettes import Spectral11, Set1_9

from lxml import objectify
from datetime import datetime
import requests
URL='http://10.0.1.99/all.xml'

output_notebook()

Loading BokehJS ...

In [2]:
Spectral11 = Set1_9
numlines=4
mypalette=Spectral11[0:numlines]

default_x = list(range(0,10))
default_y = [1,] * 10

pitp = figure(width=1000, height=500, x_axis_type="datetime", y_range=DataRange1d(start=0))

pitr = pitp.line(x = default_x, y = default_y, legend="Pit", color=Spectral11[0], line_width=2)
food1r = pitp.line(x = default_x, y = default_y, legend="Food1", color=Spectral11[1], line_width=2)
pit_targetr = pitp.line(x = default_x, y = default_y, legend="Pit Target", color=Spectral11[2], line_width=2)
food1_targetr = pitp.line(x = default_x, y = default_y, legend="Food1 Target", color=Spectral11[3], line_width=2)

pitp.xaxis.axis_label = "Time"
pitp.yaxis.axis_label = "Temperature (C)"
pitp.legend.location = "top_left"

fanp = figure(width=1000, height=100,
                 y_range = Range1d(0,100), x_axis_type="datetime")
fanr = fanp.line(x = list(range(0,10)),
                 y = [1,] * 10,
                 #line_color=mypalette,
                line_width=2)
fanp.yaxis.axis_label = "Speed (%)"

p = Column(pitp, fanp)

In [3]:
show(p, notebook_handle=True)

In [4]:
def f_to_c(f):
    return int(((float(f/10) - 32) / 1.8))
    #return f

def cyberq(filename):    
    with open(filename,'w') as f:
        writer = csv.writer(f)
        while True:
            readings = objectify.fromstring(requests.get(URL).text)
            now = datetime.now()

            raw_row = [now,
                   f_to_c(int(readings['COOK']['COOK_TEMP'])),
                   f_to_c(int(readings['COOK']['COOK_SET'])),
                   f_to_c(int(readings['FOOD1']['FOOD1_TEMP'] if readings['FOOD1']['FOOD1_TEMP'] != 'OPEN' else 0)),
                   f_to_c(int(readings['FOOD1']['FOOD1_SET'] if readings['FOOD1']['FOOD1_SET'] != 'OPEN' else 0)),
                   f_to_c(int(readings['FOOD2']['FOOD2_TEMP'] if readings['FOOD2']['FOOD2_TEMP'] != 'OPEN' else 0)),
                   f_to_c(int(readings['FOOD2']['FOOD2_SET']if readings['FOOD2']['FOOD2_SET'] != 'OPEN' else 0 )),
                   f_to_c(int(readings['FOOD3']['FOOD3_TEMP'] if readings['FOOD3']['FOOD3_TEMP'] != 'OPEN' else 0)),
                   f_to_c(int(readings['FOOD3']['FOOD3_SET']if readings['FOOD3']['FOOD3_SET'] != 'OPEN' else 0 )),
                   int(readings['OUTPUT_PERCENT'])]
            
            row = copy.deepcopy(raw_row)
            row[0] = row[0].timestamp()
            #row[0] = row[0].strftime('%Y/%m/%d %H:%M')
            writer.writerow(row)
            
            yield raw_row

def previous(filename):
    with open(filename,'r') as f:
        reader = csv.reader(f)
        
        for row in reader:
            row[0] = datetime.fromtimestamp(float(row[0]))
            yield row

def update(readings):
    (now, pit, pit_target, food1, food1_target, food2, food2_target, food3, food3_target, fan) = readings
        
    lines['pit'][now] = pit
    lines['pit_target'][now] = pit_target
    lines['food1'][now] = food1
    lines['food1_target'][now] = food1_target
    lines['food2'][now] = food2
    lines['food2_target'][now] = food2_target
    lines['food3'][now] = food3
    lines['food3_target'][now] = food3_target 
    lines['fan'][now] = fan
    
    #for reading in lines.keys():
    #    lines[reading] = lines[reading].asfreq('1s', method='pad')
        
    for r in [pitr, food1r, pit_targetr, food1_targetr]:
        r.data_source.data['x'] = lines['pit'].index.values
    
    pitr.data_source.data['y'] = lines['pit'].values
    food1r.data_source.data['y'] = lines['food1'].values
    food1_targetr.data_source.data['y'] = lines['food1_target'].values
    pit_targetr.data_source.data['y'] = lines['pit_target'].values
    
    fanr.data_source.data['x'] = lines['fan'].index.values
    fanr.data_source.data['y'] = lines['fan'].values
    
    push_notebook()

In [5]:
lines = {
    'pit': pd.Series(),
    'pit_target': pd.Series(),
    'food1': pd.Series(),
    'food1_target': pd.Series(),
    'food2': pd.Series(),
    'food2_target': pd.Series(),
    'food3': pd.Series(),
    'food3_target': pd.Series(),
    'fan': pd.Series()
}



In [8]:
for i in cyberq('lamb1.csv'):
    update(i)
    time.sleep(30)

KeyboardInterrupt: 

In [ ]:
for i in previous('test_cook.csv'):
    update(i)
    time.sleep(1)